In [1]:
test_index = 0

In [2]:
from load_data import *

In [3]:
# load_data()

In [4]:
from load_data import *

In [5]:
X_train,X_test,y_train,y_test = load_data()

In [6]:
len(X_train),len(y_train)

(843, 843)

In [7]:
len(X_test),len(y_test)

(280, 280)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [9]:
class Test_Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.c1 = nn.Conv2d(1,64,5)
        self.c2 = nn.Conv2d(64,128,5)
        self.c3 = nn.Conv2d(128,256,5)
        self.fc4 = nn.Linear(256*10*10,256)
        self.fc6 = nn.Linear(256,128)
        self.fc5 = nn.Linear(128,4)
    
    def forward(self,X):
        preds = F.max_pool2d(F.relu(self.c1(X)),(2,2))
        preds = F.max_pool2d(F.relu(self.c2(preds)),(2,2))
        preds = F.max_pool2d(F.relu(self.c3(preds)),(2,2))
#         print(preds.shape)
        preds = preds.view(-1,256*10*10)
        preds = F.relu(self.fc4(preds))
        preds = F.relu(self.fc6(preds))
        preds = self.fc5(preds)
        return preds

In [10]:
device = torch.device('cuda')

In [11]:
BATCH_SIZE = 32

In [12]:
IMG_SIZE = 112

In [13]:
model = Test_Model().to(device)
optimizer = optim.SGD(model.parameters(),lr=0.1)
criterion = nn.CrossEntropyLoss()

In [14]:
EPOCHS = 12

In [15]:
from tqdm import tqdm

In [16]:
PROJECT_NAME = 'Weather-Clf'

In [17]:
import wandb

In [18]:
test_index += 1
wandb.init(project=PROJECT_NAME,name=f'test-{test_index}')
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,1,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        preds.to(device)
        loss = criterion(preds,torch.tensor(y_batch,dtype=torch.long))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item()})
wandb.finish()